<p align="center">
  <img width="760" height="400" src="https://miro.medium.com/max/1838/1*oB3S5yHHhvougJkPXuc8og.gif">
</p>

## INTRODUCTION
### 1. Dataset
**Dogs vs. Cats** [dataset](https://www.kaggle.com/c/dogs-vs-cats/data) provided by  Microsoft Research contains 25,000 images of dogs and cats with the labels 
* 1 = dog
* 0 = cat 


# Data Preparation

* The dataset contains 12,500 images of cats and 12,500 images of dogs.
* Now we need to split this data into train and validation

    * Create a train and validation folders
    * In each folder, create cats and dogs folders
    * Move the first 10,000 images to the train folder (from 0 to 9999) for boths cats and dogs - and put them in respective folders
    * Move the remaining 2,500 images to the validation folder (from 10000 to 12499)

In [2]:
# unzip train dataset
!unzip -q '../input/dogs-vs-cats/train.zip'

# command to remove folder
#!rm -rf '/content/test'
#!rm -rf '/content/train'

# create folders for train part
!mkdir './train/cats'
!mkdir './train/dogs'

# move first 10000 images of cats and dogs to train folders
!bash -c 'mv ./train/cat.{0..9999}.jpg ./train/cats'
!bash -c 'mv ./train/dog.{0..9999}.jpg ./train/dogs'

# create folders for test part
!mkdir './validation'
!mkdir './validation/cats'
!mkdir './validation/dogs'

# move remainders to test folders
!bash -c 'mv ./train/cat.{10000..12499}.jpg ./validation/cats'
!bash -c 'mv ./train/dog.{10000..12499}.jpg ./validation/dogs'

# Loading Libraries

In [3]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD, Adam

In [5]:
# Some Cats from Training Folder
plt.figure(figsize= (18,18))
for i in range(20):
    plt.subplot(16,16,i+1)
    image = load_img("./train/cats/cat."+str(i)+'.jpg' )
    plt.axis('off')
    plt.imshow(image)

In [6]:
# Some Dogs from Training Folder
plt.figure(figsize= (18,18))
for i in range(20):
    plt.subplot(16,16,i+1)
    image = load_img("./train/dogs/dog."+str(i)+'.jpg' )
    plt.axis('off')
    plt.imshow(image)

# Setting up Model
* For this homework we will use Convolutional Neural Network (CNN. Like in the lectures, we'll use Keras.
* You need to develop the model with following structure:
    * The shape for input should be (150, 150, 3)
    * Next, create a covolutional layer (Conv2D):
        * Use 32 filters
        * Kernel size should be (3, 3) (that's the size of the filter)
        * Use 'relu' as activation
* Reduce the size of the feature map with max pooling (MaxPooling2D)
    * Set the pooling size to (2, 2)
    * Turn the multi-dimensional result into vectors using a Flatten layer
* Next, add a Dense layer with 64 neurons and 'relu' activation
* Finally, create the Dense layer with 1 neuron - this will be the output

* The output layer should have an activation - use the appropriate activation for the binary classification case
* As optimizer use SGD with the following parameters:
    * SGD(lr=0.002, momentum=0.8)

In [7]:
def model_CNN():
    model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(filters= 32, kernel_size= (3,3), activation = 'relu', input_shape = (150,150,3)),
    # """Set the pooling size to (2, 2)""""
    tf.keras.layers.MaxPooling2D((2,2)),

    # Turn the multi-dimensional result into vectors using a Flatten layer
    tf.keras.layers.Flatten(),

    # Next, add a Dense layer with 64 neurons and 'relu' activation
    tf.keras.layers.Dense( 64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

    optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()
  
    model.compile(optimizer = optimizer, 
                loss = loss,
                metrics = ['accuracy'])
    return model

In [8]:
model = model_CNN()
model.summary()

# Question 1
* Since we have a binary classification problem, what is the best loss function for us?

    * **Answer: BinaryCrossentropy**

Note: since we specify an activation for the output layer, we don't need to set from_logits=True

# Question 2

* What's the total number of parameters of the model? You can use the summary method for that.

    * **Answer: 11,215,873**

# Generators and Training

* For the next two questions, use the following data generator for both train and validation:
    * ImageDataGenerator(rescale=1./255)
* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?
    * Use batch_size=20

In [9]:
train_gen = ImageDataGenerator(rescale = 1./255)
train_generator = train_gen.flow_from_directory(
    './train/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20
)

In [10]:
val_gen = ImageDataGenerator(rescale = 1./255)
validation_generator = val_gen.flow_from_directory(
    './validation/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20
)

In [11]:
%%time
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

In [13]:
np.median(history.history['accuracy'])

# Question 3
* What is the median of training accuracy for this model?
    * **Answer: 0.596 (closest is 0.56)**

In [14]:
np.std(history.history['loss'])

# Question 4
* What is the standard deviation of training loss for this model?
    * **Answer: 0.01**

# Data Augmentation
* For the next two questions, we'll generate more data using data augmentations.
* Add the following augmentations to your training data generator:
        ```
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        ```

In [15]:
train_gen = ImageDataGenerator(
                               rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest'
)

train_generator = train_gen.flow_from_directory(
    './train/',
    target_size=(150, 150),
    classes= ['dogs', 'cats'],
    class_mode= 'binary',
    batch_size=20,
)

* **Let's train our model for 10 more epochs using the same code as previously.**
* Make sure you don't re-create the model - we want to continue training the model we already started training.

In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

In [17]:
np.mean(history.history['val_loss'])

# Question 5
* What is the mean of validation loss for the model trained with augmentations?
    * **Answer: 0.656 (closest is 0.67)**

In [19]:
np.mean(history.history['val_accuracy'][-5:])

# Question 6
* What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
* **Answer: 0.65 (close)**

In [20]:
history.history['val_accuracy'][-5:]